# FEniCS simulation of a cantilever beam

The aim of this notebook is to setup a very simple FEniCS simulation. The framework is linear, plane strain elasticity. We model a cantilever beam ($L$: total length; $d$: depth), subjected to its own weight ($\rho$: mass density per unit volume; $g$: gravity).

The beam is homogenous and isotropic ($E$: Young modulus; $\nu$: Poisson ratio) and clamped on the left end. Hooke's law reads

\begin{equation}
\sigma_{ij} = \lambda\varepsilon_{kk}\delta_{ij}+2\mu\varepsilon_{ij},
\end{equation}

where indices $i, j, k$ are restricted to $\{1, 2\}$ and $\lambda$, $\mu$ are the Lamé coefficients :

\begin{equation*}
\mu=\frac{E}{2\bigl(1+\nu\bigr)}
\quad\text{and}\quad
\lambda=\frac{2\mu\nu}{1-2\nu}.
\end{equation*}

NB: this is plane-stress.

**Question:** how does Hooke's law read in *plane stress* elasticity?

The variational formulation of the problem is the following:

Find $u\in \mathcal{C}\equiv\{u: H^1(\Omega), \; u(x_1=0,x_2)=0\}$ such that 
$\forall v\in \mathcal{C}_0\equiv \mathcal{C}$


\begin{equation}
\int_\Omega \sigma(\varepsilon(u)):\varepsilon(v)\,\mathrm{d}x\,\mathrm{d}y =
-\int_{\Omega}\rho g\,e_y\cdot v\,\mathrm{d} x\,\mathrm{d} y,
\end{equation}

where $\sigma(\varepsilon)$ is the constitutive equation and $\varepsilon(u)=\mathrm{sym} (\nabla u)$  

Remark: we work with only one unknown vector field, the displacement $\{u_x(x_1,x_2),u_y(x_1,x_2)\}$

![image of beam](cantilever_beam.png "Cantilever beam")

In [1]:
import dolfinx # FEM in python
#import matplotlib.pyplot as plt
import ufl # variational formulations
import numpy as np
#from mpi4py import MPI
import mpi4py

import basix.ufl
import dolfinx.fem as fem
import dolfinx.fem.petsc  # noqa: F401
import dolfinx.io as io
import dolfinx.mesh as mesh
import dolfinx.plot as plot
import ufl
from dolfinx import fem, io, mesh, plot, geometry

This tutorial

https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity_code.html

is in 3D. Please adapt it in the present 2D case.

## 1) Start with defining the mesh

In [2]:
# geometry and mesh 
L = 1. # total length
d = L/10. # thickness
h = d/10. # size of a cell

use 
```
dolfinx.mesh.create_rectangle()


with comm=mpi4py.MPI.COMM_WORLD
```
call your mesh : my_domain

don't forget to export your mesh in paraview...

## 2) Choose an element (Lagrange of degree 1)

define
```
V=fem.functionspace(...)
```

## 3) Define the stress

$u_i$ is  the displacement vector (note: $u=u(x_1,x_2)$)

and 

$\epsilon_{ij} = \frac12\bigl(\partial_i u_j+\partial_j u_i\bigr)$ is the stress tensor.

In [3]:
# use def to define the fonction epsilon = epsilon(u), just like
def fonction(x):
    return x**2

## 4) Write down Hooke's law

In [1]:
# here we use the values
E = 1.0
nu = 0.3

# Hooke's law is written as the top of this notebook
def sigma(epsilon):
    return XXXXXXXXX

## 5) Name `u` the `TrialFunction` and `u_bar` the `TestFunction`

## 6) Introduce the bilinear form
\begin{equation}
\int_\Omega \sigma(\varepsilon(u)):\varepsilon(\bar{u})\,\mathrm{d}x\,\mathrm{d}y
\end{equation}

use `ufl.inner( )`

## 7) Introduce the linear form

\begin{equation}
-\int_{\Omega}\rho g\,e_y\cdot \bar{v}\,\mathrm{d} x\,\mathrm{d} y.
\end{equation}

In [4]:
# here we use the value
g=0.01

## 8)  Boundary conditions

Dirichelet Boundary Condition: $u(0,x_2)=0 \quad  \forall x_2 \in [-\frac{d}{2},\frac{d}{2}]$

As $x_1=0$ is not well defined in our mesh, we write 

$u(x_1,x_2)=0 \quad \forall x_1 \simeq 0  \text{ and } \forall x_2 \in [-\frac{d}{2},\frac{d}{2}]$


In [36]:
def clamped_boundary(x):
    return np.isclose(x[0], 0)

In [37]:
# the mesh is 2D (x,y), so the boundary is 1D
facets_dim = my_domain.topology.dim - 1

In [38]:
# this finds all the facets of the elements that are at the boundary
boundary_facets = 

In [ ]:
# this gives the label of all the facets on the x_1 = 0 edge
print(boundary_facets)

In [40]:
# this finds the label of the degree of freedom for the nodes on the boundary facets
boundary_dofs = 

In [41]:
print(boundary_dofs)

[405 406 412 417 421 424 426]


In [42]:
# the Dirichlet condition
u_D = 

In [43]:
bc = dolfinx.fem.dirichletbc(u_D, boundary_dofs, V)

## 9) Solving the discretized problem

In [44]:
problem = 
# the solution is 'u_solution'
u_solution = problem.solve()

## 9.5) Verifications

In [ ]:
E, nu, g, L, d , int(L/h), int(d/h)

In [ ]:
unorm = u_solution.x.norm() # carefull, this directly (scaling is nbr_elem^0.5) depends on the number of elements
print("Solution vector norm:", unorm)
print('for E = 1.0, nu = 0.3, g=0.01, L=1.0, d=0.1, and 100x10 cells, this norm should approximately be equal to 21.5')

Compute $L^2(\Omega)$ norm:
$$ \int_\Omega u \cdot u ~\mathrm{d}x$$

In [ ]:
def assemble_scalar(J: ufl.form.Form | dolfinx.fem.Form) -> np.floating | np.complexfloating:
    """Assemble a scalar form and gather result across processes

    Args:
        form: The form to assemble.

    Returns:
        The accumulated value of the assembled form.
    """
    compiled_form = dolfinx.fem.form(J)
    if (rank := compiled_form.rank) != 0:
        raise ValueError(f"Form must be a scalar form, got for of arity {rank}")
    local_result = dolfinx.fem.assemble_scalar(compiled_form)
    return compiled_form.mesh.comm.allreduce(local_result, op=mpi4py.MPI.SUM)
    

In [ ]:
# L2-norm of the solution
print('for E = 1.0, nu = 0.3, g=0.01, L=1.0, d=0.1, and 100x10 cells, this L2-norm should approximately be equal to 0.04')
assemble_scalar(dolfinx.fem.form(ufl.dot(u_solution,u_solution) * ufl.dx))

In [ ]:
# upper-right corner of the beam
print('for E = 1.0, nu = 0.3, g=0.01, L=1.0, d=0.1, and 100x10 cells, the displacement u should approxymatly be equal to (0.08,-1.3)')
# use the function data_over_line from notebook "Poisson"

## 10) Ploting the solution

In [46]:
# To have a name in Paraview
u_solution.name = "displacement_vector"

In [47]:
# We export the mesh+solution to see it on Paraview
with dolfinx.io.XDMFFile(my_domain.comm, "out/solution.xdmf", "w") as file:
    file.write_mesh(my_domain)
    file.write_function(u_solution)

## 11) Ploting the solution using matplotlib

Recycle function data_over_line from notebook Poisson to plot vertical displacement along a horizontal line for $y=0$.

In [ ]:
# plot it
plt.plot(, ...)

## 12) We compare this u_y value to linear beam theory
$$H(x_1)=?= u_{y,fenicsX}(x_1,0)$$

$$ H(x_1) = H_\text{max} \, \left ( - \frac{1}{3} \, (x_1-1)^4 - \frac{4}{3} \, x_1 + \frac{1}{3} \right) $$

In [ ]:
# we introduce the coefficients
D=E*d**3/12./(1.-nu**2) # bending rigidity of an elastic plate
k=0.5 # unknown coefficient that we want to find (it is not equal to 0.5)
# you have to find the correct value for k !
Hmax = k*g*d/D # maximum vertical deflection, from linear 'beam' theory 

In [ ]:
Hmax

In [ ]:
x = np.linspace(0., L, num=150)

plt.plot(liste_x, # here liste fenics u_y #, "o", linewidth=2, label="FEniCSx")

H_formula = Hmax * (-1./3.*(x-1.)**4 - 4./3.*x + 1/3)
plt.plot(x, H_formula, label='Beam theory')
plt.xlabel('x',color='red',fontsize=15)
plt.ylabel('H(x,0)',color='red',fontsize=15)
plt.legend()

### Does this work, why ?

## 14) Using PVista

In [22]:
import pyvista
pyvista.start_xvfb()
pyvista.OFF_SCREEN = False
pyvista.set_jupyter_backend("static")

In [23]:
# Create pyvista grid
topology, cell_types, geometry = dolfinx.plot.vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

In [32]:
# Create plotter
plotter = pyvista.Plotter()

In [ ]:
# we first add the grid
plotter.add_mesh(grid, show_edges=True, style="wireframe", color="k");
# Then we display the scene
plotter.show()

```
Can you add the solution ?
```